In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import torch
import json

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_dir = 'drive/MyDrive/neuro2voc/task_4/data/'

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

Using device: cpu



# 1 Load everything

In [5]:
spk_t_1 = np.load(data_dir+'0_raw/spiketimes_raw_g0_t0.nidq.bin.npy')[0]
spk_t_2 = np.load(data_dir+'0_raw/spiketimes_raw_g1_t0.nidq.bin.npy')[0]
spk_t_3 = np.load(data_dir+'0_raw/spiketimes_raw_g2_t0.nidq.bin.npy')[0]
spk_cluster_1 = np.load(data_dir+'0_raw/spikecluster_raw_g0_t0.nidq.bin.npy')[0]
spk_cluster_2 = np.load(data_dir+'0_raw/spikecluster_raw_g1_t0.nidq.bin.npy')[0]
spk_cluster_3 = np.load(data_dir+'0_raw/spikecluster_raw_g2_t0.nidq.bin.npy')[0]
annotations_df_1 = pd.read_csv(data_dir+'0_raw/annotations_raw_g0_t0.nidq.bin.csv')
annotations_df_2 = pd.read_csv(data_dir+'0_raw/annotations_raw_g1_t0.nidq.bin.csv')
annotations_df_3 = pd.read_csv(data_dir+'0_raw/annotations_raw_g2_t0.nidq.bin.csv')

In [6]:
unique_cluster_1 = np.unique(spk_cluster_1).tolist()
unique_cluster_2 = np.unique(spk_cluster_2).tolist()
unique_cluster_3 = np.unique(spk_cluster_3).tolist()
assert unique_cluster_1 == unique_cluster_2
assert unique_cluster_2 == unique_cluster_3
print(len(unique_cluster_1))
print(unique_cluster_1)

75
[12.0, 19.0, 20.0, 21.0, 25.0, 27.0, 31.0, 34.0, 35.0, 36.0, 38.0, 39.0, 40.0, 41.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 72.0, 73.0, 81.0, 85.0, 92.0, 106.0, 107.0, 109.0, 110.0, 111.0, 284.0, 287.0, 291.0, 293.0, 298.0, 307.0, 311.0, 315.0, 319.0, 326.0, 327.0, 330.0, 346.0, 356.0, 370.0, 377.0, 385.0, 389.0, 396.0, 401.0, 403.0, 406.0, 411.0, 416.0, 419.0, 425.0, 428.0]


In [7]:
unique_cluster_1_sorted = sorted(np.unique(unique_cluster_1))
neuron2idx = {neuron_name: idx for idx, neuron_name in enumerate(unique_cluster_1_sorted)}
print(neuron2idx)

{12.0: 0, 19.0: 1, 20.0: 2, 21.0: 3, 25.0: 4, 27.0: 5, 31.0: 6, 34.0: 7, 35.0: 8, 36.0: 9, 38.0: 10, 39.0: 11, 40.0: 12, 41.0: 13, 44.0: 14, 45.0: 15, 46.0: 16, 47.0: 17, 48.0: 18, 49.0: 19, 50.0: 20, 51.0: 21, 52.0: 22, 54.0: 23, 55.0: 24, 56.0: 25, 57.0: 26, 58.0: 27, 59.0: 28, 60.0: 29, 61.0: 30, 63.0: 31, 64.0: 32, 65.0: 33, 66.0: 34, 67.0: 35, 68.0: 36, 69.0: 37, 72.0: 38, 73.0: 39, 81.0: 40, 85.0: 41, 92.0: 42, 106.0: 43, 107.0: 44, 109.0: 45, 110.0: 46, 111.0: 47, 284.0: 48, 287.0: 49, 291.0: 50, 293.0: 51, 298.0: 52, 307.0: 53, 311.0: 54, 315.0: 55, 319.0: 56, 326.0: 57, 327.0: 58, 330.0: 59, 346.0: 60, 356.0: 61, 370.0: 62, 377.0: 63, 385.0: 64, 389.0: 65, 396.0: 66, 401.0: 67, 403.0: 68, 406.0: 69, 411.0: 70, 416.0: 71, 419.0: 72, 425.0: 73, 428.0: 74}


In [8]:
with open(data_dir+'0_raw/idx2depth.json', 'r') as f:
    idx2depth = json.load(f)
print(idx2depth)

{'0': 660, '1': 1320, '2': 1320, '3': 1340, '4': 1680, '5': 1820, '6': 2140, '7': 2220, '8': 2240, '9': 2260, '10': 2280, '11': 2300, '12': 2320, '13': 2340, '14': 2380, '15': 2360, '16': 2400, '17': 2420, '18': 2440, '19': 2440, '20': 2460, '21': 2480, '22': 2480, '23': 2500, '24': 2520, '25': 2520, '26': 2540, '27': 2540, '28': 2580, '29': 2580, '30': 2580, '31': 2600, '32': 2600, '33': 2600, '34': 2600, '35': 2640, '36': 2640, '37': 2660, '38': 2680, '39': 2720, '40': 2880, '41': 2960, '42': 3200, '43': 2380, '44': 2560, '45': 2640, '46': 2660, '47': 2720, '48': 2740, '49': 2580, '50': 2580, '51': 2460, '52': 2680, '53': 840, '54': 480, '55': 480, '56': 2860, '57': 200, '58': 3220, '59': 3180, '60': 2980, '61': 1060, '62': 1120, '63': 1060, '64': 1080, '65': 1680, '66': 1880, '67': 2100, '68': 2160, '69': 2180, '70': 2280, '71': 2360, '72': 2700, '73': 2820, '74': 3380}


In [9]:
# create tuples (neuron_name, depth)
neuron_depth_list = [(neuron_name, idx2depth[str(neuron2idx[neuron_name])]) for neuron_name in unique_cluster_1_sorted]
print(neuron_depth_list)

[(12.0, 660), (19.0, 1320), (20.0, 1320), (21.0, 1340), (25.0, 1680), (27.0, 1820), (31.0, 2140), (34.0, 2220), (35.0, 2240), (36.0, 2260), (38.0, 2280), (39.0, 2300), (40.0, 2320), (41.0, 2340), (44.0, 2380), (45.0, 2360), (46.0, 2400), (47.0, 2420), (48.0, 2440), (49.0, 2440), (50.0, 2460), (51.0, 2480), (52.0, 2480), (54.0, 2500), (55.0, 2520), (56.0, 2520), (57.0, 2540), (58.0, 2540), (59.0, 2580), (60.0, 2580), (61.0, 2580), (63.0, 2600), (64.0, 2600), (65.0, 2600), (66.0, 2600), (67.0, 2640), (68.0, 2640), (69.0, 2660), (72.0, 2680), (73.0, 2720), (81.0, 2880), (85.0, 2960), (92.0, 3200), (106.0, 2380), (107.0, 2560), (109.0, 2640), (110.0, 2660), (111.0, 2720), (284.0, 2740), (287.0, 2580), (291.0, 2580), (293.0, 2460), (298.0, 2680), (307.0, 840), (311.0, 480), (315.0, 480), (319.0, 2860), (326.0, 200), (327.0, 3220), (330.0, 3180), (346.0, 2980), (356.0, 1060), (370.0, 1120), (377.0, 1060), (385.0, 1080), (389.0, 1680), (396.0, 1880), (401.0, 2100), (403.0, 2160), (406.0, 2180

In [10]:
sorted_neuron_depth_list = sorted(neuron_depth_list, key=lambda x: x[1])
print(sorted_neuron_depth_list)

[(326.0, 200), (311.0, 480), (315.0, 480), (12.0, 660), (307.0, 840), (356.0, 1060), (377.0, 1060), (385.0, 1080), (370.0, 1120), (19.0, 1320), (20.0, 1320), (21.0, 1340), (25.0, 1680), (389.0, 1680), (27.0, 1820), (396.0, 1880), (401.0, 2100), (31.0, 2140), (403.0, 2160), (406.0, 2180), (34.0, 2220), (35.0, 2240), (36.0, 2260), (38.0, 2280), (411.0, 2280), (39.0, 2300), (40.0, 2320), (41.0, 2340), (45.0, 2360), (416.0, 2360), (44.0, 2380), (106.0, 2380), (46.0, 2400), (47.0, 2420), (48.0, 2440), (49.0, 2440), (50.0, 2460), (293.0, 2460), (51.0, 2480), (52.0, 2480), (54.0, 2500), (55.0, 2520), (56.0, 2520), (57.0, 2540), (58.0, 2540), (107.0, 2560), (59.0, 2580), (60.0, 2580), (61.0, 2580), (287.0, 2580), (291.0, 2580), (63.0, 2600), (64.0, 2600), (65.0, 2600), (66.0, 2600), (67.0, 2640), (68.0, 2640), (109.0, 2640), (69.0, 2660), (110.0, 2660), (72.0, 2680), (298.0, 2680), (419.0, 2700), (73.0, 2720), (111.0, 2720), (284.0, 2740), (425.0, 2820), (319.0, 2860), (81.0, 2880), (85.0, 296

In [11]:
sorted_neuron2idx = {neuron_name: idx for idx, (neuron_name, _) in enumerate(sorted_neuron_depth_list)}
print("New neuron2idx ordered by depth:", sorted_neuron2idx)

New neuron2idx ordered by depth: {326.0: 0, 311.0: 1, 315.0: 2, 12.0: 3, 307.0: 4, 356.0: 5, 377.0: 6, 385.0: 7, 370.0: 8, 19.0: 9, 20.0: 10, 21.0: 11, 25.0: 12, 389.0: 13, 27.0: 14, 396.0: 15, 401.0: 16, 31.0: 17, 403.0: 18, 406.0: 19, 34.0: 20, 35.0: 21, 36.0: 22, 38.0: 23, 411.0: 24, 39.0: 25, 40.0: 26, 41.0: 27, 45.0: 28, 416.0: 29, 44.0: 30, 106.0: 31, 46.0: 32, 47.0: 33, 48.0: 34, 49.0: 35, 50.0: 36, 293.0: 37, 51.0: 38, 52.0: 39, 54.0: 40, 55.0: 41, 56.0: 42, 57.0: 43, 58.0: 44, 107.0: 45, 59.0: 46, 60.0: 47, 61.0: 48, 287.0: 49, 291.0: 50, 63.0: 51, 64.0: 52, 65.0: 53, 66.0: 54, 67.0: 55, 68.0: 56, 109.0: 57, 69.0: 58, 110.0: 59, 72.0: 60, 298.0: 61, 419.0: 62, 73.0: 63, 111.0: 64, 284.0: 65, 425.0: 66, 319.0: 67, 81.0: 68, 85.0: 69, 346.0: 70, 330.0: 71, 92.0: 72, 327.0: 73, 428.0: 74}


In [12]:
syllable_df_1 = annotations_df_1[annotations_df_1['cluster_id'].isin(range(2, 10))].copy()
syllable_df_2 = annotations_df_2[annotations_df_2['cluster_id'].isin(range(2, 10))].copy()
syllable_df_3 = annotations_df_3[annotations_df_3['cluster_id'].isin(range(2, 10))].copy()

In [13]:
unique_labels = syllable_df_3['cluster_id'].unique()
label2idx = {label: idx for idx, label in enumerate(unique_labels)}
label2idx

{2: 0, 3: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6}

# 2 Extract the tensors

In [14]:
def map_label2idx(label, mapping):
    '''
    used for both label2idx (for singing syllables) and for neuron2idx (for neuron clusters)
    '''
    return mapping.get(label, -1)

def stripe_data2tensor(annotations_df, spk_t, spk_cluster, label2idx, neuron2idx, directory, file, num_neurons=75, time_before_onset=100, time_after_onset=100):
    # Don't map 2-8 to 0-6
    # annotations_df['idx'] = annotations_df['cluster_id'].apply(lambda x: map_label2idx(x, label2idx))

    # initialize lists
    data = []
    labels = []

    # iterate through annotations
    for _, annotation in tqdm(annotations_df.iterrows(), total=annotations_df.shape[0], desc="Processing entries"):
        onset = annotation['onset']
        start_time = int(onset - time_before_onset)
        end_time = int(onset + time_after_onset)
        label_idx = annotation['cluster_id']

        # initialize tensor for this entry
        num_time_points = end_time - start_time
        entry_tensor = torch.zeros((num_neurons, num_time_points), dtype=torch.int)

        # create a time mask for all spikes in the time window
        time_mask = (spk_t >= start_time) & (spk_t < end_time)

        # filter spike times and locations based on the time mask
        filtered_spk_t = spk_t[time_mask]
        filtered_spk_cluster = spk_cluster[time_mask]

        # iterate over this pair of filtered time and space info, write into tensor
        for spike_time, neuron_idx in zip(filtered_spk_t, filtered_spk_cluster):

            # find the time for this spike relative to the start_time (onset-time_before_onset)
            time_within_entry = int(spike_time - start_time)
            mapped_neuron_idx = map_label2idx(neuron_idx, neuron2idx)

            if mapped_neuron_idx != -1:
                # document the firing in time_within_entry
                entry_tensor[mapped_neuron_idx, time_within_entry] = 1
            else:
                print(f"Error, the mapped neuron idx is {mapped_neuron_idx}.")
                return None

        data.append(entry_tensor)
        labels.append(label_idx)

    # stack tensors and convert labels to tensor
    print("Stacking tensors...")
    data_tensor = torch.stack(data)
    labels_tensor = torch.tensor(labels)

    # dimension check
    expected_time_points = time_before_onset + time_after_onset
    if len(labels) != data_tensor.shape[0] or \
       num_neurons != data_tensor.shape[1] or \
       expected_time_points != data_tensor.shape[2]:
        print("Error: Dimension mismatch in final firing patterns tensor.")
        print(f"length of labels: {len(labels)}")
        print(f"data tensor shape 0: {data_tensor.shape[0]}")
        print(f"num_neurons: {num_neurons}")
        print(f"data tensor shape 1: {data_tensor.shape[1]}")
        print(f"expected_time_points: {expected_time_points}")
        print(f"data tensor shape 2: {data_tensor.shape[2]}")

    print("Saving tensors...")
    # save the tensors
    tensor_dir = os.path.join(directory, "1_tensors/")
    os.makedirs(tensor_dir, exist_ok=True)
    data_path_name = os.path.join(tensor_dir, f"data_{file}_{time_before_onset}_{time_after_onset}.pt")
    labels_path_name = os.path.join(tensor_dir, f"labels_{file}_{time_before_onset}_{time_after_onset}.pt")
    torch.save(data_tensor, data_path_name)
    torch.save(labels_tensor, labels_path_name)
    print(f"Data saved to {data_path_name}")
    print(f"Labels saved to {labels_path_name}")

    return data_tensor, labels_tensor

### 750-750

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=750, time_after_onset=750)

Processing entries: 100%|██████████| 417/417 [00:05<00:00, 83.28it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_1_750_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_1_750_750.pt


In [ ]:
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=750, time_after_onset=750)

Processing entries: 100%|██████████| 191/191 [00:02<00:00, 67.45it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_2_750_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_2_750_750.pt


In [ ]:
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=750, time_after_onset=750)

Processing entries: 100%|██████████| 2851/2851 [00:48<00:00, 58.75it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_3_750_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_3_750_750.pt


500 - 750

In [ ]:
before = 500
after = 750

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:04<00:00, 87.03it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_1_500_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_1_500_750.pt


Processing entries: 100%|██████████| 191/191 [00:02<00:00, 67.84it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_2_500_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_2_500_750.pt


Processing entries: 100%|██████████| 2851/2851 [00:55<00:00, 51.35it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_3_500_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_3_500_750.pt


In [ ]:
before = 500
after = 500

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:04<00:00, 94.21it/s] 


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_1_500_500.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_1_500_500.pt


Processing entries: 100%|██████████| 191/191 [00:02<00:00, 70.84it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_2_500_500.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_2_500_500.pt


Processing entries: 100%|██████████| 2851/2851 [00:48<00:00, 59.17it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_3_500_500.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_3_500_500.pt


In [ ]:
before = 500
after = 1000

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:06<00:00, 67.75it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_1_500_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_1_500_1000.pt


Processing entries: 100%|██████████| 191/191 [00:02<00:00, 69.91it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_2_500_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_2_500_1000.pt


Processing entries: 100%|██████████| 2851/2851 [00:49<00:00, 58.08it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_3_500_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_3_500_1000.pt


In [ ]:
before = 400
after = 600

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:04<00:00, 103.53it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_1_400_600.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_1_400_600.pt


Processing entries: 100%|██████████| 191/191 [00:03<00:00, 61.89it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_2_400_600.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_2_400_600.pt


Processing entries: 100%|██████████| 2851/2851 [00:47<00:00, 60.60it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_3_400_600.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_3_400_600.pt


In [ ]:
before = 400
after = 800

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:04<00:00, 99.47it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_1_400_800.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_1_400_800.pt


Processing entries: 100%|██████████| 191/191 [00:03<00:00, 61.07it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_2_400_800.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_2_400_800.pt


Processing entries: 100%|██████████| 2851/2851 [00:49<00:00, 57.56it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_3_400_800.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_3_400_800.pt


In [ ]:
before = 400
after = 1000

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:04<00:00, 99.21it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_1_400_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_1_400_1000.pt


Processing entries: 100%|██████████| 191/191 [00:02<00:00, 66.59it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_2_400_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_2_400_1000.pt


Processing entries: 100%|██████████| 2851/2851 [00:49<00:00, 57.84it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_3_400_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_3_400_1000.pt


In [ ]:
before = 750
after = 1000

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:04<00:00, 95.55it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_1_750_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_1_750_1000.pt


Processing entries: 100%|██████████| 191/191 [00:03<00:00, 58.28it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_2_750_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_2_750_1000.pt


Processing entries: 100%|██████████| 2851/2851 [00:52<00:00, 54.31it/s]


Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_3_750_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_3_750_1000.pt


In [ ]:
before = 750
after = 500

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:03<00:00, 106.01it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_1_750_500.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_1_750_500.pt


Processing entries: 100%|██████████| 191/191 [00:02<00:00, 76.02it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_2_750_500.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_2_750_500.pt


Processing entries: 100%|██████████| 2851/2851 [00:43<00:00, 65.43it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_3_750_500.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_3_750_500.pt


In [ ]:
before = 2000
after = 1000

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:06<00:00, 64.08it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_2000_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_2000_1000.pt


Processing entries: 100%|██████████| 191/191 [00:04<00:00, 43.82it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_2000_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_2000_1000.pt


Processing entries: 100%|██████████| 2851/2851 [01:03<00:00, 44.97it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_2000_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_2000_1000.pt


In [ ]:
before = 2000
after = 750

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:06<00:00, 65.80it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_2000_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_2000_750.pt


Processing entries: 100%|██████████| 191/191 [00:03<00:00, 48.01it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_2000_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_2000_750.pt


Processing entries: 100%|██████████| 2851/2851 [01:18<00:00, 36.18it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_2000_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_2000_750.pt


In [ ]:
before = 2000
after = 1250

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:07<00:00, 56.40it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_2000_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_2000_1250.pt


Processing entries: 100%|██████████| 191/191 [00:05<00:00, 32.82it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_2000_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_2000_1250.pt


Processing entries: 100%|██████████| 2851/2851 [01:07<00:00, 42.02it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_2000_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_2000_1250.pt


In [ ]:
before = 1500
after = 1000

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:05<00:00, 73.10it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_1500_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_1500_1000.pt


Processing entries: 100%|██████████| 191/191 [00:04<00:00, 46.40it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_1500_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_1500_1000.pt


Processing entries: 100%|██████████| 2851/2851 [01:02<00:00, 45.63it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_1500_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_1500_1000.pt


In [ ]:
before = 1500
after = 750

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:04<00:00, 87.51it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_1500_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_1500_750.pt


Processing entries: 100%|██████████| 191/191 [00:03<00:00, 49.47it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_1500_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_1500_750.pt


Processing entries: 100%|██████████| 2851/2851 [00:58<00:00, 48.35it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_1500_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_1500_750.pt


In [ ]:
before = 1250
after = 1250

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:04<00:00, 89.40it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_1250_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_1250_1250.pt


Processing entries: 100%|██████████| 191/191 [00:03<00:00, 50.56it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_1250_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_1250_1250.pt


Processing entries: 100%|██████████| 2851/2851 [00:58<00:00, 49.10it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_1250_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_1250_1250.pt


In [ ]:
before = 1250
after = 1000

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:04<00:00, 88.10it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_1250_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_1250_1000.pt


Processing entries: 100%|██████████| 191/191 [00:03<00:00, 48.24it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_1250_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_1250_1000.pt


Processing entries: 100%|██████████| 2851/2851 [00:57<00:00, 49.70it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_1250_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_1250_1000.pt


In [ ]:
before = 1250
after = 750

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:04<00:00, 91.41it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_1250_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_1250_750.pt


Processing entries: 100%|██████████| 191/191 [00:03<00:00, 53.98it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_1250_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_1250_750.pt


Processing entries: 100%|██████████| 2851/2851 [00:54<00:00, 52.50it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_1250_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_1250_750.pt


In [ ]:
before = 1000
after = 750

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:05<00:00, 78.83it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_1000_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_1000_750.pt


Processing entries: 100%|██████████| 191/191 [00:03<00:00, 57.79it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_1000_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_1000_750.pt


Processing entries: 100%|██████████| 2851/2851 [00:52<00:00, 54.75it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_1000_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_1000_750.pt


In [ ]:
before = 1000
after = 1250

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:05<00:00, 83.04it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_1000_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_1000_1250.pt


Processing entries: 100%|██████████| 191/191 [00:04<00:00, 41.75it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_1000_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_1000_1250.pt


Processing entries: 100%|██████████| 2851/2851 [00:59<00:00, 48.23it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_1000_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_1000_1250.pt


In [ ]:
before = 1000
after = 1000

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:04<00:00, 85.94it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_1000_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_1000_1000.pt


Processing entries: 100%|██████████| 191/191 [00:03<00:00, 59.16it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_1000_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_1000_1000.pt


Processing entries: 100%|██████████| 2851/2851 [00:56<00:00, 50.56it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_1000_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_1000_1000.pt


# 4 Do a moving window

In [ ]:
before = 0
after = 1000

In [ ]:
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:05<00:00, 76.17it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_1_0_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_1_0_1000.pt


Processing entries: 100%|██████████| 191/191 [00:02<00:00, 67.57it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_2_0_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_2_0_1000.pt


Processing entries: 100%|██████████| 2851/2851 [00:49<00:00, 57.64it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/data_3_0_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_3/data/1_tensors/labels_3_0_1000.pt


# More data

In [15]:
before = 0
after = 750
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:05<00:00, 72.55it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_0_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_0_750.pt


Processing entries: 100%|██████████| 191/191 [00:03<00:00, 62.21it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_0_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_0_750.pt


Processing entries: 100%|██████████| 2851/2851 [00:47<00:00, 59.40it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_0_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_0_750.pt


In [16]:
before = 0
after = 1250
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:05<00:00, 79.10it/s] 


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_0_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_0_1250.pt


Processing entries: 100%|██████████| 191/191 [00:03<00:00, 63.18it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_0_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_0_1250.pt


Processing entries: 100%|██████████| 2851/2851 [00:59<00:00, 48.10it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_0_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_0_1250.pt


In [17]:
before = 400
after = 750
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:04<00:00, 91.67it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_400_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_400_750.pt


Processing entries: 100%|██████████| 191/191 [00:03<00:00, 56.23it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_400_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_400_750.pt


Processing entries: 100%|██████████| 2851/2851 [00:56<00:00, 50.89it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_400_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_400_750.pt


In [18]:
before = 400
after = 1250
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:05<00:00, 69.80it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_400_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_400_1250.pt


Processing entries: 100%|██████████| 191/191 [00:05<00:00, 34.34it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_400_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_400_1250.pt


Processing entries: 100%|██████████| 2851/2851 [00:56<00:00, 50.29it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_400_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_400_1250.pt


In [19]:
before = 750
after = 1250
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:04<00:00, 89.46it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_750_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_750_1250.pt


Processing entries: 100%|██████████| 191/191 [00:05<00:00, 35.15it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_750_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_750_1250.pt


Processing entries: 100%|██████████| 2851/2851 [00:57<00:00, 49.57it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_750_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_750_1250.pt


In [20]:
before = 1250
after = 1000
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:05<00:00, 82.01it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_1250_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_1250_1000.pt


Processing entries: 100%|██████████| 191/191 [00:03<00:00, 52.29it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_1250_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_1250_1000.pt


Processing entries: 100%|██████████| 2851/2851 [00:57<00:00, 49.80it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_1250_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_1250_1000.pt


In [21]:
before = 750
after = 1250
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:04<00:00, 89.48it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_750_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_750_1250.pt


Processing entries: 100%|██████████| 191/191 [00:04<00:00, 43.77it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_750_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_750_1250.pt


Processing entries: 100%|██████████| 2851/2851 [00:59<00:00, 48.02it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_750_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_750_1250.pt


In [22]:
before = 1750
after = 750
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:11<00:00, 36.09it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_1750_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_1750_750.pt


Processing entries: 100%|██████████| 191/191 [00:07<00:00, 27.28it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_1750_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_1750_750.pt


Processing entries: 100%|██████████| 2851/2851 [00:56<00:00, 50.14it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_1750_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_1750_750.pt


In [23]:
before = 1750
after = 1000
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:05<00:00, 72.16it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_1750_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_1750_1000.pt


Processing entries: 100%|██████████| 191/191 [00:04<00:00, 47.64it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_1750_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_1750_1000.pt


Processing entries: 100%|██████████| 2851/2851 [01:06<00:00, 42.69it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_1750_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_1750_1000.pt


In [24]:
before = 1750
after = 1250
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:05<00:00, 79.13it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_1750_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_1750_1250.pt


Processing entries: 100%|██████████| 191/191 [00:05<00:00, 32.70it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_1750_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_1750_1250.pt


Processing entries: 100%|██████████| 2851/2851 [01:11<00:00, 40.03it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_1750_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_1750_1250.pt


In [25]:
before = 2250
after = 750
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:05<00:00, 80.25it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_2250_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_2250_750.pt


Processing entries: 100%|██████████| 191/191 [00:04<00:00, 41.64it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_2250_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_2250_750.pt


Processing entries: 100%|██████████| 2851/2851 [01:11<00:00, 40.07it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_2250_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_2250_750.pt


In [26]:
before = 2250
after = 1000
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:06<00:00, 65.87it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_2250_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_2250_1000.pt


Processing entries: 100%|██████████| 191/191 [00:07<00:00, 24.21it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_2250_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_2250_1000.pt


Processing entries: 100%|██████████| 2851/2851 [01:08<00:00, 41.72it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_2250_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_2250_1000.pt


In [27]:
before = 2250
after = 1250
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:06<00:00, 66.95it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_2250_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_2250_1250.pt


Processing entries: 100%|██████████| 191/191 [00:05<00:00, 38.09it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_2250_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_2250_1250.pt


Processing entries: 100%|██████████| 2851/2851 [01:13<00:00, 38.71it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_2250_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_2250_1250.pt


In [28]:
before = 2500
after = 750
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:06<00:00, 65.43it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_2500_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_2500_750.pt


Processing entries: 100%|██████████| 191/191 [00:03<00:00, 50.97it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_2500_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_2500_750.pt


Processing entries: 100%|██████████| 2851/2851 [01:15<00:00, 37.76it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_2500_750.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_2500_750.pt


In [29]:
before = 2500
after = 1000
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:06<00:00, 65.52it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_2500_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_2500_1000.pt


Processing entries: 100%|██████████| 191/191 [00:05<00:00, 32.06it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_2500_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_2500_1000.pt


Processing entries: 100%|██████████| 2851/2851 [01:12<00:00, 39.53it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_2500_1000.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_2500_1000.pt


In [30]:
before = 2500
after = 1250
data_tensor_1, labels_tensor_1 = stripe_data2tensor(syllable_df_1, spk_t_1, spk_cluster_1, label2idx, sorted_neuron2idx, directory=data_dir, file="1", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_2, labels_tensor_2 = stripe_data2tensor(syllable_df_2, spk_t_2, spk_cluster_2, label2idx, sorted_neuron2idx, directory=data_dir, file="2", num_neurons=75, time_before_onset=before, time_after_onset=after)
data_tensor_3, labels_tensor_3 = stripe_data2tensor(syllable_df_3, spk_t_3, spk_cluster_3, label2idx, sorted_neuron2idx, directory=data_dir, file="3", num_neurons=75, time_before_onset=before, time_after_onset=after)

Processing entries: 100%|██████████| 417/417 [00:05<00:00, 74.56it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_1_2500_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_1_2500_1250.pt


Processing entries: 100%|██████████| 191/191 [00:05<00:00, 37.31it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_2_2500_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_2_2500_1250.pt


Processing entries: 100%|██████████| 2851/2851 [01:13<00:00, 39.03it/s]


Stacking tensors...
Saving tensors...
Data saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/data_3_2500_1250.pt
Labels saved to drive/MyDrive/neuro2voc/task_4/data/1_tensors/labels_3_2500_1250.pt
